In [ ]:
import numpy as np
from skopt import gp_minimize

def f(x):
    return (np.sin(5 * x[0]) * (1 - np.tanh(x[0] ** 2)) +
            np.random.randn() * 0.1)

res = gp_minimize(f, [(-2.0, 2.0)])

res

In [ ]:
from skopt import Optimizer

opt = Optimizer([(-2.0, 2.0)], base_estimator='gp')

for i in range(20):
    suggested = opt.ask()
    y = f(suggested)
    opt.tell(suggested, y)
    print('iteration:', i, suggested, y)

Minimize with scipy

In [ ]:
import numpy as np
from scipy.optimize import minimize
import datetime as datetime
import price_utils as pu

time = [
 datetime.datetime(2014, 1, 1, 0, 0), datetime.datetime(2014, 1, 1, 1, 0),
 datetime.datetime(2014, 1, 1, 2, 0), datetime.datetime(2014, 1, 1, 3, 0),
 datetime.datetime(2014, 1, 1, 4, 0), datetime.datetime(2014, 1, 1, 5, 0),
 datetime.datetime(2014, 1, 1, 6, 0), datetime.datetime(2014, 1, 1, 7, 0),
 datetime.datetime(2014, 1, 1, 8, 0), datetime.datetime(2014, 1, 1, 9, 0),
 datetime.datetime(2014, 1, 1, 10, 0), datetime.datetime(2014, 1, 1, 11, 0),
 datetime.datetime(2014, 1, 1, 12, 0), datetime.datetime(2014, 1, 1, 13, 0),
 datetime.datetime(2014, 1, 1, 14, 0), datetime.datetime(2014, 1, 1, 15, 0),
 datetime.datetime(2014, 1, 1, 16, 0), datetime.datetime(2014, 1, 1, 17, 0),
 datetime.datetime(2014, 1, 1, 18, 0), datetime.datetime(2014, 1, 1, 19, 0),
 datetime.datetime(2014, 1, 1, 20, 0), datetime.datetime(2014, 1, 1, 21, 0),
 datetime.datetime(2014, 1, 1, 22, 0), datetime.datetime(2014, 1, 1, 23, 0)
 ]

cons = np.array([
  3.50201667,  3.54595    , 3.57833333,  3.52298333,  3.5632   ,   3.47963333,
  3.54918333 , 3.03461667 , 0.38033333 , 0.40003333 , 0.17285   ,  0.2043,
  0.18611667,  0.16973333 , 0.35173333,  0.20706667 , 0.23841667,  0.75256667,
  3.69731667,  3.83086667 , 3.80273333 , 3.77096667 , 3.88846667 , 3.77421667
  ])

prod = np.array([
  -0.01341667, -0.01356667, -0.01353333, -0.01336667 ,-0.01346667, -0.01328333,
 -0.01348333 , 0.06991667 , 1.22971667 , 2.66196667 , 3.46321667 , 2.31033333,
  1.57818333 , 2.2803    ,  2.31488333 , 0.7913    ,  0.44245    , 0.05665,
 -0.01345    ,-0.01308333, -0.013      ,-0.01316667, -0.013      ,-0.01341667
 ])

prices = np.array(pu.get_prices_per_hour(time))


# Normalizovanie prod na 0 alebo kladne
prod = list(map(lambda x: max(0, x), prod))




def f(x):
  batt_charge = [0] * len(x)
  grid = [0] * len(x)
  for idx,w in enumerate(x):
    batt_charge[idx] = batt_charge[idx-1] + prod[idx]
    batt_charge[idx] = min(10, batt_charge[idx])
    
    pred = batt_charge[idx]
    po = batt_charge[idx] * w

    z_baterie = pred - po

    grid[idx] += max(0, cons[idx] - z_baterie)
    batt_charge[idx] = po

  return sum(grid*prices)

x0 = np.array([1] * len(prod))

res = minimize(f, x0, method='L-BFGS-B',  options={'xtol': 1e-8, 'disp': False, 'maxiter':5000})
print(res.x)

